In [ ]:
from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

from datetime import datetime
import numpy as np

# load data
train_samples_radio1 = np.load('train_samples_radio1.npy')
train_samples_radio2 = np.load('train_samples_radio2.npy')
train_samples_radio3 = np.load('train_samples_radio3.npy')
train_samples_radio4 = np.load('train_samples_radio4.npy')

train_labels_radio1 = np.load('train_labels_radio1.npy')
train_labels_radio2 = np.load('train_labels_radio2.npy')
train_labels_radio3 = np.load('train_labels_radio3.npy')
train_labels_radio4 = np.load('train_labels_radio4.npy')

test_samples_radio1 = np.load('test_samples_radio1.npy')
test_samples_radio2 = np.load('test_samples_radio2.npy')
test_samples_radio3 = np.load('test_samples_radio3.npy')
test_samples_radio4 = np.load('test_samples_radio4.npy')

test_labels_radio1 = np.load('test_labels_radio1.npy')
test_labels_radio2 = np.load('test_labels_radio2.npy')
test_labels_radio3 = np.load('test_labels_radio3.npy')
test_labels_radio4 = np.load('test_labels_radio4.npy')

# concatenate data
Xtrain = np.concatenate((train_samples_radio1,
                         train_samples_radio2,
                         train_samples_radio3,
                         train_samples_radio4))

Ytrain = np.concatenate((train_labels_radio1,
                         train_labels_radio2,
                         train_labels_radio3,
                         train_labels_radio4))

Xtest = np.concatenate((test_samples_radio1,
                        test_samples_radio2,
                        test_samples_radio3,
                        test_samples_radio4))

Ytest = np.concatenate((test_labels_radio1,
                        test_labels_radio2,
                        test_labels_radio3,
                        test_labels_radio4))

# define our model name
model_name = "radio-identification"+datetime.now().strftime("%Y%m%d-%H%M%S")

# Define our network architecture:

# Input data is of the shape 32x32  
network = input_data(shape=[None, 32, 32, 1])

network = conv_2d(network, 128, 5, activation='prelu')
network = conv_2d(network, 96, 1, activation='prelu')
network = conv_2d(network, 64, 1, activation='prelu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 128, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 128, 5, activation='prelu')
network = conv_2d(network, 128, 1, activation='prelu')
network = conv_2d(network, 128, 1, activation='prelu')
network = avg_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 64, activation='tanh')
network = dropout(network, 0.8)

#Fully-connected neural network with 4 outputs (radio1, radio2, radio3, radio4) to make the final prediction
network = fully_connected(network, 4, activation='softmax')

# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='/tmp/tflearn_logs/'+model_name+'.tfl.ckpt')

# Train it! We'll do 100 training passes and monitor it as it goes.
model.fit(Xtrain, Ytrain, n_epoch=50, shuffle=True, validation_set=(Xtest, Ytest),
          show_metric=True, batch_size=245,
          snapshot_epoch=True,
          run_id=model_name)

# Save model when training is complete to a file

model.save(model_name+'.tfl')
print("Network trained and saved as: "+model_name+".tfl")